In [2]:
import numpy as np
import pandas as pd

In [3]:
# read data into dataframe
df = pd.read_csv("OnlineNewsPopularity.csv")

In [4]:
# drop unused features
vals = [0, 1]
for i in range(13, 39):
    vals.append(i)

df = df.drop(df.columns[vals], axis = 1)
df.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [19]:
# get mean vector 
# mean for each attribute
mean_vector = df.mean()
print(mean_vector)


 n_tokens_title                    10.398749
 n_tokens_content                 546.514731
 n_unique_tokens                    0.548216
 n_non_stop_words                   0.996469
 n_non_stop_unique_tokens           0.689175
 num_hrefs                         10.883690
 num_self_hrefs                     3.293638
 num_imgs                           4.544143
 num_videos                         1.249874
 average_token_length               4.548239
 num_keywords                       7.223767
 LDA_00                             0.184599
 LDA_01                             0.141256
 LDA_02                             0.216321
 LDA_03                             0.223770
 LDA_04                             0.234029
 global_subjectivity                0.443370
 global_sentiment_polarity          0.119309
 global_rate_positive_words         0.039625
 global_rate_negative_words         0.016612
 rate_positive_words                0.682150
 rate_negative_words                0.287934
 avg_posit

In [38]:
# store dataframe as 2D array
matrix = df.to_numpy()

var = []
vec = []

# iterate over every row in the column
# build the column data into a vector
for i in range(len(matrix)):
    vec.append(matrix[i][0])

#calculate the total variance of the attribute
d = np.dot(vec, vec)
(d / len(matrix)) - (mean_vector[0]*mean_vector[0])

4.469038894755826